<a href="https://colab.research.google.com/github/Seiris21/2022_snowpack_capstone/blob/main/notebooks/data_ingestion/SnowCast%20Showdown%20Data%20Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%capture
!apt-get update
# Install GDAL and Geopandas
!apt-get install libgdal-dev 
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet

%pip install -U tornado

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
%pip install "dask[complete]"
%pip install "dask[complete]" --upgrade

In [ ]:
#%%capture
%pip install pystac_client planetary_computer rasterio xarray-spatial

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt

from google.colab import files
from datetime import datetime

from pystac_client import Client
import planetary_computer
import xarray
import dask.dataframe as dd
import xrspatial
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation
from datashader.utils import export_image

# Data Ingestion and Exploration

In [ ]:
trainfeatures = pd.read_csv("/content/drive/MyDrive/snowcapstone team spring 2022/Competition_Data/ground_measures_train_features.csv")

In [ ]:
trainfeatures.head()

In [ ]:
trainfeatures = trainfeatures.melt(id_vars=['Unnamed: 0']).dropna().reset_index(drop = True)
trainfeatures.rename(columns = {'Unnamed: 0':"station_id", "variable":"date", "value":"SWE"}, inplace = True)
trainfeatures.head()

In [ ]:
#These are dates where no stations had information (Discovered after using KNN approach)
#Alternative method: Get all dates of cell_id samples, and compare those dates against what each station has for the interpolation
nan_dates = ['2013-04-03', '2013-04-29', '2013-05-03', '2013-05-25', '2013-06-01', '2013-06-08', '2016-02-08', '2016-03-26', '2016-04-01', '2016-04-03', '2016-04-04',
 '2016-04-07', '2016-04-16', '2016-05-09', '2016-05-27', '2016-06-26', '2017-01-28', '2017-01-29', '2018-03-04', '2018-03-30', '2018-03-31', '2018-04-22', '2018-04-23', 
 '2018-04-25', '2018-04-26', '2018-05-24', '2018-05-28', '2018-06-01', '2018-06-02', '2019-03-09', '2019-03-15', '2019-03-16', '2019-03-17', '2019-03-24', '2019-03-25', 
 '2019-03-29', '2019-04-07', '2019-04-08', '2019-04-17', '2019-04-18', '2019-04-19', '2019-04-21', '2019-04-27', '2019-04-28', '2019-05-01', '2019-05-02', '2019-05-03', 
 '2019-06-05', '2019-06-08', '2019-06-09', '2019-06-10', '2019-06-13', '2019-06-14', '2019-06-24']

In [ ]:
#Linear regression implementation, filling in based on nan_dates list
supplement = []

#Iterate through all the unique stations
for station in trainfeatures['station_id'].unique(): #['CDEC:SSM']: #
  #Get subset for this station
  subset = trainfeatures[trainfeatures['station_id']==station].copy()
  #make filler rows with missing dates
  filler = [[station,date,np.nan] for date in nan_dates]
  #Append filler rows to subset and sort on date and reset index
  subset = subset.append(pd.DataFrame(filler, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)
  #print(station,len(subset.index))
  #print(subset.head())
  for date in nan_dates:
    #Find NaN date
    nan_index = subset.index[subset['date'] == date].tolist()[0]
    nan_date = datetime.strptime(date,'%Y-%m-%d')
    
    #There is a conditional needed for stations that stopped reporting before 2019
    try:
      count=0
      #Find older date that HAS value. Sometimes needed because filler inserted NaNs
      while subset.iloc[nan_index-1-count].isnull().any():
        count+=1
      #Older date (nan-1)
      if (nan_index-1-count)>=0:
        older_date = datetime.strptime(subset.iloc[nan_index-1-count]['date'],'%Y-%m-%d')
        older_swe = subset.iloc[nan_index-1-count]['SWE']
      else:
        older_date = datetime.strptime(subset.iloc[nan_index-1]['date'],'%Y-%m-%d')
        older_swe = np.nan
      #print('Older',nan_index-1,older_date,older_swe)
      #print('NaN-inserted',nan_index,nan_date)

      #Newer date is next date that HAS value, otherwise enter except
      counter=0
      while subset.iloc[nan_index+1+counter].isnull().any():
        counter+=1
      #Newer date
      newer_date = datetime.strptime(subset.iloc[nan_index+1+counter]['date'],'%Y-%m-%d')
      newer_swe = subset.iloc[nan_index+1+counter]['SWE']
      #print('newer',nan_index+1+counter,newer_date,newer_swe)
      #print('______________________________')

      #Change per day
      delta_day = (newer_swe-older_swe)/(newer_date-older_date).days

      #Add expected change to older swe
      est_swe = older_swe + (delta_day*(nan_date-older_date).days)

      #Add "entry" row to supplement
      supplement.append([station,date,est_swe])
    #IndexError happens when the last date is actually from the nan list. Because of this, We DEFINITELY need to do some inter-station interpolation
    except IndexError:
      supplement.append([station,date,np.nan])

#Problem with simple linear interpolation: There are large enough gaps that the "missing days" in the data sometimes are the closest dates to themselves

In [ ]:
#Add incomplete supplement (testing to see how many knn nans are filled in)
trainfeatures = trainfeatures.append(pd.DataFrame(supplement, columns=['station_id','date','SWE'])).sort_values(by='date').reset_index(drop=True)

In [ ]:
trainmeta = pd.read_csv("/content/drive/MyDrive/snowcapstone team spring 2022/Competition_Data/ground_measures_metadata.csv")
trainmeta.head()

In [ ]:
trainfeatures = trainfeatures.merge(trainmeta, how = 'left', on='station_id')
trainfeatures.head()

In [ ]:
gridcells = gpd.read_file('/content/drive/MyDrive/snowcapstone team spring 2022/Competition_Data/grid_cells.geojson')
print(gridcells.head())

In [ ]:
traindf = pd.read_csv("/content/drive/MyDrive/snowcapstone team spring 2022/Competition_Data/train_labels.csv")

traindf = traindf.melt(id_vars=["cell_id"]).dropna().reset_index(drop = True)
traindf.rename(columns = {"cell_id":"cell_id", "variable":"date", "value":"SWE"}, inplace = True)

traindf = traindf.merge(gridcells, how = 'left', on='cell_id')


traindf = gpd.GeoDataFrame(traindf, crs ="EPSG:4326")
traindf

In [ ]:
from datetime import datetime, timedelta

print(len(traindf.loc[pd.to_datetime(traindf.date) > datetime.strptime("2016-01-01", "%Y-%m-%d"), "region"]))
print(len(traindf.loc[traindf["region"] == "sierras", "region"]))

In [ ]:
print(len(traindf))
traindf = traindf.loc[pd.to_datetime(traindf.date) >= datetime.strptime("2016-01-01", "%Y-%m-%d")].reset_index(drop = True)
print(len(traindf["region"]))
traindf.head()

In [ ]:
gdf = gpd.GeoDataFrame(trainmeta, 
                       geometry = gpd.points_from_xy(trainmeta.longitude, trainmeta.latitude),
                       crs = "EPSG:4326")

gdf

In [ ]:
import matplotlib.pyplot as plt

#fig, ax = plt.subplots(figsize = (240,240))
#
#states = gpd.read_file('/content/drive/MyDrive/snowcapstone team spring 2022/Competition_Data/USMap/cb_2018_us_state_20m.shp')
#states = states.to_crs("EPSG:4326")
#states = states[states['STUSPS'].isin(['WA', 'OR', 'CA', 'NV', 'MT', 'ID', 'WY', 'NM', 'CO' ,'UT', 'AZ'])]
#statemap = states.boundary.plot(ax=ax, linewidth=5, zorder = 1)
#
#gpd.GeoDataFrame(traindf["geometry"]).to_crs(states.crs).plot(ax=ax, facecolor="none", edgecolor='grey')
#
#gdf["geometry"].plot(ax = ax, markersize = 200, color = 'red',marker = '*', zorder = 2)
#
#plt.autoscale(False)
#ax.axis("off")
#
#plt.show()

# Adding Station Data

In this section we will take the measurements of ground stations and add those as features to our data



In [ ]:
trainfeatures = trainfeatures[['station_id',	'date',	'SWE',	'name']]

### Balltree/KNN approach

In [ ]:
from sklearn.neighbors import BallTree

#Adapted from AutoGIS| University of Helsinki
# https://automating-gis-processes.github.io/site/notebooks/L3/nearest-neighbor-faster.html
def get_knearest(src_points, candidates, knn=1):
  '''
  K nearest neighbors for every source point given candidate points
  '''
  #Make candidates BallTree format
  tree = BallTree(candidates,leaf_size=15,metric='haversine')

  #Find closest points
  distances, indices = tree.query(src_points, k=knn)

  #Transpose into arrays
  distances = distances.transpose()
  indices = indices.transpose()

  #neighbor_idx = []
  #neighbor_dist = []
   
  return(indices, distances)
  #Iterate for k neighbors
  #for i in range(knn):
  #  neighbor_idx.append(indices[i])
  #  neighbor_dist.append(distances[i])
  #Return list of lists in order of KNN
  #return(neighbor_idx,neighbor_dist)



  return distances,indices

def nearest_neighbor(left_gdf, right_gdf, return_dist=False, knn=1):
  """
  For each point in left_gdf, find closest point in right GeoDataFrame and return them.

  NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
  """
  #Some Nan buffer to KNN search
  knn = knn*3

  left_geom_col = left_gdf.geometry.name
  right_geom_col = right_gdf.geometry.name

  # Ensure that index in right gdf is formed of sequential numbers
  right = right_gdf.copy().reset_index(drop=True)

  # Parse coordinates from points and insert them into a numpy array as RADIANS
  # For left radians, data is in polygon format, so apply meter crs, get centroid, and revert
  left_radians = np.array(left_gdf[left_geom_col].to_crs('epsg:4087').centroid.to_crs("EPSG:4326").apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
  right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())


  # Find the nearest points
  # -----------------------
  # closest ==> index in right_gdf that corresponds to the closest point
  # dist ==> distance between the nearest neighbors (in meters)

  closest, dist = get_knearest(src_points=left_radians, candidates=right_radians, knn=knn)

  #return(closest,dist)
    
  closest_points = gpd.GeoDataFrame()
    
  #Loop for knn
  for i in range(knn):
    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    #Loop to return closest starting from 0 idx
    closest_points['station_id_'+str(i)] = right['station_id'].loc[closest[i]].values
    closest_points['elevation_m_'+str(i)] = right['elevation_m'].loc[closest[i]].values

    # Add distance if requested
    if return_dist:
      # Convert to meters from radians
      earth_radius = 6371000  # meters
      closest_points['distance_'+str(i)] = dist[i] * earth_radius

  return closest_points

def inverseDmean(df,power):
  #Formula for inverse Distance Average = ((x1/d1^p)+(x2/d2^p)....)/((1/d1^p)+(1/d2^p)....)
  #https://gisgeography.com/inverse-distance-weighting-idw-interpolation/
  subset = df.filter(regex='distance_[0-9]+|SWE_[0-9]+')
  numerator = pd.DataFrame()
  denominator = pd.DataFrame()

  for i in range(int(subset.shape[1]/2)):
    numerator['x_'+str(i)]=subset['SWE_'+str(i)]/(subset['distance_'+str(i)]**power)
    denominator['x_'+str(i)]=1/(subset['distance_'+str(i)]**power)

  #There are cells without SWE data. We do not want this in the inverse Distance Calculation
  nulls = np.where(pd.isnull(numerator))
  for row,column in zip(nulls[0],nulls[1]):
    denominator.at[row, denominator.columns[column]] = np.nan
  
  numerator['sum']=numerator.sum(axis=1)
  #print(numerator.head())
  denominator['sum']=denominator.sum(axis=1)
  #print(denominator.head())

  return(numerator['sum']/denominator['sum'])

def swe_calculation(train, labels, closest_stations, knn=1):
  #Join labels with closest_stations
  labels_joined = labels.join(closest_stations)

  #Prepare column names
  SWE_names=[]
  elevation_names=[]
  reordered_columns = ['cell_id', 'date', 'SWE', 'region', 'geometry',
                       'mean_inversed_swe', 'mean_local_swe',	'median_local_swe',	'max_local_swe', 'min_local_swe',
                       'mean_local_elevation',	'median_local_elevation',	'max_local_elevation','min_local_elevation']
  for i in range(knn):
    reordered_columns.extend(['station_id_'+str(i),'elevation_m_'+str(i),'distance_'+str(i),'SWE_'+str(i)])
    SWE_names.append('SWE_'+str(i))
    elevation_names.append('elevation_m_'+str(i))
  
  #Merge against cell_id+date to get closest stations for each cell
  idx = 0
  for i in range(knn*3):
    train
    if i == 0:
      tmp_merged = pd.merge(labels_joined, train, how="left", left_on=['station_id_'+str(i), 'date'], right_on=['station_id','date'],suffixes=(None,'_'+str(i))).drop(columns= ['station_id'])
    else:
      tmp_merged = pd.merge(tmp_merged, train, how="left", left_on=['station_id_'+str(i), 'date'], right_on=['station_id','date'],suffixes=(None,'_'+str(i))).drop(columns= ['station_id'])

  #Filter out nearest neighbors with NaN, get 5 closest WITH VALUES
  filtered = []
  for idx,row in tmp_merged.iterrows():
    index = []
    values = []
    i=0
    counter=0
    while i<knn:
      if not pd.isna(row['SWE_'+str(counter)]):
        i+=1
        index.append(counter)
      counter+=1
    for j in index:
      values.extend([row['station_id_'+str(j)], row['elevation_m_'+str(j)], row['distance_'+str(j)], row['SWE_'+str(j)]])
    filtered.append(values)

  #Re-merge with cell data
  merged_train = labels.join(pd.DataFrame(filtered,columns=reordered_columns[-4*knn:]))

  #Calculations
  #Elevations
  # Normal Mean
  merged_train['mean_local_elevation']=merged_train[elevation_names].mean(axis=1)
  # Median
  merged_train['median_local_elevation']=merged_train[elevation_names].median(axis=1)
  # Max
  merged_train['max_local_elevation']=merged_train[elevation_names].max(axis=1)
  # Min
  merged_train['min_local_elevation']=merged_train[elevation_names].min(axis=1)

  #SWE
  #Inverse Distance Mean
  merged_train['mean_inversed_swe']=inverseDmean(merged_train,2)
  #Normal Mean
  merged_train['mean_local_swe']=merged_train[SWE_names].mean(axis=1)
  #Median
  merged_train['median_local_swe']=merged_train[SWE_names].median(axis=1)
  #Min
  merged_train['min_local_swe']=merged_train[SWE_names].min(axis=1)
  #Max
  merged_train['max_local_swe']=merged_train[SWE_names].max(axis=1)

  #Reorder Columns
  merged_train=merged_train[reordered_columns]

  return(merged_train)

In [ ]:
knn=5
#DO NOT want traindf.date or trainfeatures in datetime format

closest_stations = nearest_neighbor(traindf, gdf, return_dist=True,knn=knn)
traindf = swe_calculation(train=trainfeatures, labels=traindf, closest_stations=closest_stations, knn=knn)

In [ ]:
traindf.sample(frac = .1).head()

In [ ]:
traindf = traindf[['cell_id','date','SWE','region','geometry','mean_inversed_swe',
                   'mean_local_swe','median_local_swe','max_local_swe','min_local_swe',
                   'mean_local_elevation','median_local_elevation','max_local_elevation','min_local_elevation',]]
                   
print(traindf.isna().sum())

# Modis Data

## Environment Setup

In [ ]:
import shutil
import os
import requests
import ee
from datetime import datetime, timedelta
import signal

class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

signal.signal(signal.SIGALRM, timeout_handler)

traindf["date"] = pd.to_datetime(traindf.date)
trainfeatures['date'] = pd.to_datetime(trainfeatures.date)

#I am creating a string version of the date to use as a filename
traindf["datestring"] = traindf.date.map(lambda d: str(d.year)+d.strftime('%j'))

#Now I calculate my centroid from the provided geometry
#Ignore the warnings this creates. It is in a projected crs
traindf["centroid"] = traindf.geometry.to_crs('+proj=cea').centroid
traindf["center_lat"] = traindf.centroid.y
traindf["center_long"] = traindf.centroid.x

#Logging in to Earth Engine
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [ ]:
'''
def pull_MODIS(traindf, modis, overwrite = False, names_only = False):
  filelocations = []
  x = 0

  for i in range(len(traindf.SWE)):

    #create a name for the image
    pict_name = traindf.cell_id[i] + '_' + modis + '_' + traindf.datestring[i] + '.jpg'

    #create the whole filename with path to the correct folder
    filename = os.path.join('/content/', modis, pict_name)

    if names_only:
      filelocations.append(filename)
      x += 1
      if x % 5000 == 0:
        print(f'{x} files already exist')

    elif os.path.exists(filename) and not overwrite:
      filelocations.append(filename)
      x += 1
      if x % 5000 == 0:
        print(f'{x} files already exist')

    else:
      #We need a start date and an end date. Just like a regular python slice, 
      #the end date is not included, so by using a 1 day frame, I am actually limiting
      #the range to only the day in question
      start_date = traindf.date[i] - timedelta(days = 7)
      end_date = traindf.date[i] + timedelta(days = 1)

      #First I get the image collection from the MODIS data, filter it only to the days in question
      #and select my bands, then sort so the most recent day in the group is at the top
      Collection = ee.ImageCollection(f'MODIS/006/{modis}') \
                  .filter(ee.Filter.date(start_date, end_date)) \
                  .filter(ee.Filter.notNull(['system:index'])) \
                  .select(['NDSI_Snow_Cover', 'Snow_Albedo_Daily_Tile', 'NDSI']) \
                  .sort('system:index', False) 

      #I create a google earth images point based on the area centroid
      centroid = ee.Geometry.Point(traindf.center_long[i], traindf.center_lat[i])

      #Because the image collection is limited to a single day, there is only one image
      #So I just take it
      point = Collection.first().unmask(0)

      # Get individual band arrays and build them into an RGB image
      # The "buffer" is a circular distance around the point, measured in meters right now it is 100km
      rgb = ee.Image.rgb(point.clip(centroid.buffer(10000)).select('NDSI_Snow_Cover').divide(100), #I divide by 100 to get it between 0 and 1
                        point.clip(centroid.buffer(10000)).select('Snow_Albedo_Daily_Tile').divide(100), #I divide by 100 to get it between 0 and 1
                        point.clip(centroid.buffer(10000)).select('NDSI').divide(10000)).visualize() #I divide by 10000 to get it between 0 and 1

      #Now I get the url for the image
      url = rgb.getThumbURL()

      #add the name to my list I created earlier
      filelocations.append(filename)

      #now I open the url and download the image to the specified file location
      response = requests.get(url, stream=True)
      with open(filename, 'wb') as out_file:
          shutil.copyfileobj(response.raw, out_file)
      del response
    
  #traindf[f"{modis}_filelocations"] = filelocations
'''

In [ ]:
def pull_MODIS_list(traindf, modis, signal_timer = 5):
  datalist = []
  x= 0

  still_working = True
  while still_working:
    try:
      Collection = ee.ImageCollection(f'MODIS/006/{modis}') \
                  .select(['NDSI_Snow_Cover', 'Snow_Albedo_Daily_Tile', 'NDSI'])
      
    except Exception as e:
      print("Some Error with Image Collection")
    else: 
      signal.alarm(0)
      still_working = False        
  

  for i in range(len(traindf.SWE)):
    still_working = True
    while still_working:
      signal.alarm(signal_timer)
      try:
        row = [traindf.cell_id[i], traindf.date[i]]

        #We need a start date and an end date. Just like a regular python slice, 
        #the end date is not included, so by using a 1 day frame, I am actually limiting
        #the range to only the day in question
        start_date = traindf.date[i] - timedelta(days = 7)
        end_date = traindf.date[i] + timedelta(days = 1)

        #First I get the image collection from the MODIS data, filter it only to the days in question
        #and select my bands, then sort so the most recent day in the group is at the top
        DatedCollection = Collection.filter(ee.Filter.date(start_date, end_date)) \
                                    .filter(ee.Filter.notNull(['system:index'])) \
                                    .sort('system:index', False)

        #Because the image collection is limited to a single day, there is only one image
        #So I just take it
        point = DatedCollection.first().unmask(0)

        aoi = ee.Geometry.Polygon(list(traindf.iloc[i].geometry.exterior.coords))

        bands = point.reduceRegion(reducer = ee.Reducer.mean(),
        geometry= aoi)

        bands = bands.toArray(['NDSI_Snow_Cover', 'Snow_Albedo_Daily_Tile', 'NDSI']).getInfo()
        bands = np.divide(bands, [100,100,10000] )

        row.extend(bands)

        datalist.append(row)
      
      except TimeoutException:
        print(f"Request Timeout for cell_id {traindf.cell_id[i]}")
      
      except Exception as e:
        print("Some other Error")
      else: 
        signal.alarm(0)
        still_working = False
        x+=1
        if x % 100 == 0:
          print(f'{x} out of {len(traindf.SWE)} complete')

  data = pd.DataFrame(datalist, columns = ['cell_id', 'date', f'{modis}_SnowCover', 
                                           f'{modis}_Albedo', f'{modis}_NDSI'])
  print(data)
  traindf.merge(data, how = 'left', on=['cell_id', 'date'])
  return traindf

In [ ]:
import time 

In [ ]:
start = time.time()
traindf = pull_MODIS_list(traindf, modis = "MOD10A1")
print(f'Total Time: {time.time() - start}')
print()

start = time.time()
traindf = pull_MODIS_list(traindf, modis = "MYD10A1")
print(f'Total Time: {time.time() - start}')


#!zip -r '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MOD10A1_sierras.zip'  '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MOD10A1/'
#!zip -r '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MYD10A1_sierras.zip'  '/content/drive/MyDrive/snowcapstone team spring 2022/MODIS_Data/MYD10A1/'

# Adding Copernicus Data

#MATCH FUNCTION

In [ ]:
def get_copernicus(traindf, overwrite = False):
  traindf["copernicus_filelocations"] = "blank"
  x = 0
  length_cell_id = len(traindf.cell_id.unique())

  for i in traindf.cell_id.unique():
    #create a name for the image
    pict_name = i + '_' + 'copernicus90m'

    #create the whole filename with path to the correct folder
    filename = os.path.join('/content/drive/MyDrive/snowcapstone team spring 2022/Copernicus_Data', pict_name)

    # Adapted from https://planetarycomputer.microsoft.com/dataset/cop-dem-glo-90#Example-Notebook :
    
    if not os.path.exists(filename + '.png') or overwrite:
      client = Client.open(
          "https://planetarycomputer.microsoft.com/api/stac/v1",
          ignore_conformance=True,
      )

      point = [traindf.loc[traindf.cell_id == i, "center_long"].iloc[0], 
              traindf.loc[traindf.cell_id == i, "center_lat"].iloc[0]]
      
      search = client.search(
          collections=["cop-dem-glo-90"],
          intersects={"type": "Point", "coordinates": point},
      )

      items = list(search.get_items())

      signed_asset = planetary_computer.sign(items[0].assets["data"])
      
      data = (
          xarray.open_rasterio(signed_asset.href)
          .squeeze()
          .drop("band")
          .mean()
      )
      min_lon = min([j[0] for j in [y for y in traindf.loc[traindf.cell_id == i, "geometry"].iloc[0].centroid.buffer(.05).boundary.coords]])
      min_lat = min([j[1] for j in [y for y in traindf.loc[traindf.cell_id == i, "geometry"].iloc[0].centroid.buffer(.05).boundary.coords]])
      max_lon = max([j[0] for j in [y for y in traindf.loc[traindf.cell_id == i, "geometry"].iloc[0].centroid.buffer(.05).boundary.coords]])
      max_lat = max([j[1] for j in [y for y in traindf.loc[traindf.cell_id == i, "geometry"].iloc[0].centroid.buffer(.05).boundary.coords]])

      mask_lon = (data.x >= min_lon) & (data.x <= max_lon)
      mask_lat = (data.y >= min_lat) & (data.x <= max_lat)

      cropped_data = data.where(mask_lon & mask_lat, drop=True)

      hillshade = xrspatial.hillshade(cropped_data)
      img = stack(shade(hillshade, cmap=["white", "gray"]), shade(cropped_data, cmap=Elevation, alpha=128))
      export_image(img=img, filename=filename, background=None)

    traindf.loc[traindf.cell_id == i, "copernicus_filelocations"] = filename + '.png'
    if x % 500 == 0:
      print(f'{x} out of {length_cell_id} complete')
    x += 1

In [ ]:
get_copernicus(traindf, False)

In [ ]:
traindf = traindf.drop(["region", 'geometry', 'datestring', 'centroid', 'center_lat', 'center_long'], axis = 1)
traindf.head()

In [ ]:
pd.DataFrame(traindf).to_csv('/content/drive/MyDrive/snowcapstone team spring 2022/Modeling/traindf_modis_columns_allregions.csv')

In [ ]:
'''
#Potential Ideas:
#Include reverse distance interpolation.
#

def get_stations(df, num_stations = 5):
  #Returns the average SWE from the X closest measurement stations on the same day
  averages = []
  average_elevation = []
  median = []
  median_elevation = []
  max = []
  max_elevation = []
  #Calculating distance matrix
  dist1 = gpd.GeoDataFrame(geometry = df.geometry.unique(), crs = "EPSG:4326").to_crs('+proj=cea')
  dist2 = gpd.GeoDataFrame(geometry = trainmeta.geometry, crs = "EPSG:4326").to_crs('+proj=cea') 

  distmatrix = dist2.geometry.apply(lambda g: dist1.geometry.boundary.distance(g))
  distmatrix.set_axis(df.cell_id.unique(), axis = 1, inplace = True)
  distmatrix["station_id"] = trainmeta.station_id.unique()
  
  #for each row in the df
  for index, row in df.iterrows():
    station_dist =distmatrix[["station_id", row.cell_id]]

    start_date = row.date - timedelta(days = 7)
    end_date = row.date
    after_start_date = trainfeatures["date"] >= start_date
    before_end_date = trainfeatures["date"] <= end_date
    between_two_dates = after_start_date & before_end_date
    swes = trainfeatures.loc[between_two_dates]

    station_swe = station_dist.merge(swes, how = 'left', on='station_id')
    station_swe = station_swe[station_swe["SWE"].notna()]

    station_swe.sort_values(by = [row.cell_id, "date"], inplace = True)

    #This is where interpolation can happen

    averages.append(station_swe.head(num_stations).SWE.mean())
    average_elevation.append(station_swe.head(num_stations).elevation_m.mean())
    median.append(station_swe.head(num_stations).SWE.median())
    median_elevation.append(station_swe.head(num_stations).elevation_m.median())
    max.append(station_swe.head(num_stations).SWE.max())
    max_elevation.append(station_swe.head(num_stations).elevation_m.max())

  #return averages, median, max, average_elevation, median_elevation, max_elevation
  return averages, average_elevation, median, median_elevation, max, max_elevation
  '''

In [ ]:
'''
#Turns off a bunch of errors
pd.options.mode.chained_assignment = None  # default='warn'

averages, average_elevation, median, median_elevation, max, max_elevation = get_stations(traindf)

pd.options.mode.chained_assignment = 'warn'
'''

In [ ]:
'''
traindf["mean_local_swe"] = averages
traindf["median_local_swe"] = median
traindf["max_local_swe"] = max
traindf["mean_local_elevation"] = average_elevation
traindf["median_local_elevation"] = median_elevation
traindf["max_local_elevation"] = max_elevation

print(traindf.isna().sum())
'''

# Ingesting Sentinel 2 Data

## Pulling in Data by date/1k square and saving as .tiff format

In [ ]:
## Test that ee connection is still live
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [ ]:
## Import additional packages (check for duplicates)
import pickle
import time
import math
import requests, zipfile, io
from IPython.display import Image

### Defining Image Collection

Define image selection to Copernicus/S2 or Sentinel 2 data. 

Available bands and resolution here: https://gisgeography.com/sentinel-2-bands-combinations/

The following visualizations are the most applicable to our study:

Geology: Bands B12, B11, B2
Vegetation: B8-B4/B8+B4

Starting with focus on geology bands, this pulls in the B12(R), B11(G), B4(B)

B12 and B11 are at 20m resolution, and B4 is at 10m resolution

### Sentinel 2

In [ ]:
#defining image
sen2 = ee.ImageCollection("COPERNICUS/S2").filterDate(startDate, endDate) 

#selecting bands
sen2 = sen2.select(["B12","B11","B2"])

### Sentinel 1


In [ ]:
#defining image

sen1 = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(startDate, endDate) 

#selecting bands
sen1_A = sen1.select(["HH","HV","angle"])
sen1_B = sen1.select(["VV", "VH", "angle"])

## Function to define image download

Create a funciton to download imagery. Takes the centroid from above and size input and pulls images into google drive. 

Len = total size of image in meters computed as the resolution of the image bands being pulled in, and the number of pixels we want to capture total (224x224). Then create bounding box around the circle to get a square. 

In [ ]:
def image_download(image,point,image_res,n_pixels,folder_name, image_name, storage="Drive"):
    
    """
    Function to download satellite images from a ee.imageCollection object.
    We first generate a bounding box of image_res*n_pixels meters around "point",
    then we clip that region from the image collection, take the mean image from the collection,
    and send that as a task to the Google Earth Engine. 
    After that, we download the image Google Cloud Storage if storage=="Cloud", 
    to Google Drive if storage=="Drive" or to a local folder if storage=="local".
    
    Inputs:
    -image= ee.ImageCollection object
    -point= ee.Geometry.Point object
    -image_res= resolution of the image in meters
    -n_pixels= number of pixels to extract on the images
    -storage= string indicating if we are storing the images in Google Cloud,Google Drive or locally.
              Defaults to local storage.
    -folder_name= string with Google Cloud bucket name if storage=="Cloud"
                  string with the name of a folder in the root of Google Drive if storage=="Drive"
                  string with the path to the image if storage=="local"
    -image_name= string with the image_name for the TIFF image.

    Output:
     When storage=="Drive":
     -task= an EE task object. we can then use task.status() to check the status of the task.
     If the task is completed, we will see a TIFF image in "folder_name" with name "image_name.tif".
     The image has 3 dimensions, where the first 2 are n_pixels, and the 3rd is the number of bands of "image".
     When storage=="local":
     -there is no output, but we will see one TIFF file per band of our image in the folder "folder_name".
    """
    #generating the box around the point
    len=image_res*n_pixels # for sen2, 20 meters * 224 pixels
    region= point.buffer(len/2).bounds().getInfo()['coordinates']
    #defining the rectangle
    coords=np.array(region)
    #taking min and maxs of coordinates to define the rectangle
    coords=[np.min(coords[:,:,0]), np.min(coords[:,:,1]), np.max(coords[:,:,0]), np.max(coords[:,:,1])]
    rectangle=ee.Geometry.Rectangle(coords)
    
    if storage=="Drive":
        #generating the export task (dimensions is "WIDTHxHEIGHT")
        task=ee.batch.Export.image.toDrive(image=image.filterBounds(rectangle).mean(), 
                            folder=folder_name, 
                            description=image_name, 
                            region=str(region), dimensions=str(n_pixels)+"x"+str(n_pixels))
        #starting the task
        task.start()
        return task
    
    if storage=="local":
        #downloading the image
        r=requests.get( image.filterBounds(rectangle).mean().getDownloadURL({
                            'name': image_name, 
                            'region': str(region),
                            'dimensions': str(n_pixels)+"x"+str(n_pixels)}))
        #unzip it to the selected directory
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(folder_name)

TEST

In [ ]:
#test image
test=ee.Geometry.Point(-120.61888999873261,39.675880337476684)

#running function
image_download(image=sen2,point=test,image_res=100,n_pixels=224,folder_name='Sen2_Tiff', image_name='test_image', storage="local")

In [ ]:
os.listdir('Sen2_Tiff')

Iterate over all of the dataframe

In [ ]:
traindf.head()

In [ ]:
for i in range(len(traindf)):

    #create a name for the image
    image_name = traindf.cell_id[i] + '_sentinel2_' + traindf.datestring[i] + '.jpg'

    
    startDate = traindf.date[i] - timedelta(days = 365)
    endDate = traindf.date[i] + timedelta(days = 1)
    year = startDate.year
    print(year)
    print(startDate)
    
      #defining image

    sen2 = ee.ImageCollection("COPERNICUS/S2")
      # filter date
    sen2 = sen2.filterDate(startDate, endDate) 
      #applying cloud masking
      #selecting bands
    sen2 = sen2.select(["B12","B11","B2"])
    
      #I create a google earth images point based on the area centroid
    centroid = ee.Geometry.Point(traindf.center_long[i], traindf.center_lat[i])
    print(centroid)
      
    image_download(image=sen2, point=centroid, image_res=20, n_pixels=224, folder_name='Sen2_Tiff', image_name='image_name', storage="local")


In [ ]:
for i in range(len(traindf)):

    #create a name for the image
    image_name = traindf.cell_id[i] + '_sentinel1a_' + traindf.datestring[i] + '.jpg'

    
    startDate = traindf.date[i] - timedelta(days = 7)
    endDate = traindf.date[i] + timedelta(days = 1)
    year = startDate.year
    print(year)
    print(startDate)
    

      #defining image

    sen1 = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(startDate, endDate) 

      #selecting bands
    sen1_A = sen1.select(["HH","HV","angle"])
      # filter date

    
      #I create a google earth images point based on the area centroid
    centroid = ee.Geometry.Point(traindf.center_long[i], traindf.center_lat[i]).buffer(10000)
    print(centroid)
      
    image_download(image=sen1_A, point=centroid, image_res=20, n_pixels=224, folder_name='Sen1a_Tiff', image_name='image_name', storage="local")


In [ ]:
for i in range(len(traindf)):

    #create a name for the image
    image_name = traindf.cell_id[i] + '_sentinel1b_' + traindf.datestring[i] + '.jpg'

    
    startDate = traindf.date[i] - timedelta(days = 7)
    endDate = traindf.date[i] + timedelta(days = 1)
    year = startDate.year
    print(year)
    print(startDate)
    


    sen1 = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(startDate, endDate) 

      #selecting bands
    sen1_B = sen1.select(["VV", "VH", "angle"])
      # filter date

    
      #I create a google earth images point based on the area centroid
    centroid = ee.Geometry.Point(traindf.center_long[i], traindf.center_lat[i])
    print(centroid)
      
    image_download(image=sen1_b, point=centroid, image_res=20, n_pixels=224, folder_name='Sen1b_Tiff', image_name='image_name', storage="local")


## Reading in .tiff images and loading max, min, medians to testdf

In [ ]:
def pull_SENT2(traindf, overwrite = False, names_only = False):
  filelocations = []
  x = 0

  for i in range(len(traindf.SWE)):

    #create a name for the image
    pict_name = traindf.cell_id[i] + '_sentinel2_' + traindf.datestring[i] + '.jpg'

    #create the whole filename with path to the correct folder
    filename = os.path.join('/content/drive/MyDrive/snowcapstone team spring 2022/Sen2_Tiff', pict_name)

    if names_only:
      filelocations.append(filename)
      x += 1
      if x % 5000 == 0:
        print(f'{x} files already exist')

    elif os.path.exists(filename) and not overwrite:
      filelocations.append(filename)
      x += 1
      if x % 5000 == 0:
        print(f'{x} files already exist')

    else:
      #We need a start date and an end date. Just like a regular python slice, 
      #the end date is not included, so by using a 1 day frame, I am actually limiting
      #the range to only the day in question
      start_date = traindf.date[i] - timedelta(days = 7)
      end_date = traindf.date[i] + timedelta(days = 1)

      #First I get the image collection from the MODIS data, filter it only to the days in question
      #and select my bands, then sort so the most recent day in the group is at the top
      Collection = ee.ImageCollection('COPERNICUS/S2') \
                  .filter(ee.Filter.date(start_date, end_date)) \
                  .select(['B11', 'B12', 'B2']) \

      #I create a google earth images point based on the area centroid
      centroid = ee.Geometry.Point(traindf.center_long[i], traindf.center_lat[i])

      #Because the image collection is limited to a single day, there is only one image
      #So I just take it
      point = Collection.first().unmask(0)

      # Get individual band arrays and build them into an RGB image
      # The "buffer" is a circular distance around the point, measured in meters right now it is 100km
      rgb = ee.Image.rgb(point.clip(centroid.buffer(10000)).select('B11'),
                        point.clip(centroid.buffer(10000)).select('B12'),
                        point.clip(centroid.buffer(10000)).select('B2'))

      #Now I get the url for the image
      url = rgb.getThumbURL({'min': -20, 'max': 0})

      #add the name to my list I created earlier
      filelocations.append(filename)

      #now I open the url and download the image to the specified file location
      response = requests.get(url, stream=True)
      with open(filename, 'wb') as out_file:
          shutil.copyfileobj(response.raw, out_file)
      del response
    
  traindf["Sentinel2_filelocations"] = filelocations

In [ ]:
pull_SENT2(traindf, overwrite = True)